訓練用**countN**，驗證用count

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
# df_2019 = pd.read_parquet('../time_series/TS6/TS6-2019.parquet', engine='pyarrow')
# df_2020 = pd.read_parquet('../time_series/TS6/TS6-2020.parquet', engine='pyarrow')
df_2021 = pd.read_parquet('../time_series/TS6/TS6-2021.parquet', engine='pyarrow')
df_2022 = pd.read_parquet('../time_series/TS6/TS6-2022.parquet', engine='pyarrow')

df = pd.concat([df_2021, df_2022], ignore_index=False)
df.head()

,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,TempTime,countN
13154400,lyft,2021,1,1,0,3,4,True,15,40.864294,-73.84651,2021-01-01 00:00:00,12.000000
13154401,lyft,2021,1,1,1,3,4,True,22,40.864294,-73.84651,2021-01-01 01:00:00,11.666667
13154402,lyft,2021,1,1,2,3,4,True,17,40.864294,-73.84651,2021-01-01 02:00:00,11.333333
13154403,lyft,2021,1,1,3,3,4,True,16,40.864294,-73.84651,2021-01-01 03:00:00,8.333333
13154404,lyft,2021,1,1,4,3,4,True,12,40.864294,-73.84651,2021-01-01 04:00:00,8.333333


In [3]:
df.shape

(13718160, 13)

In [4]:
df = df.drop(['year', 'TempTime'], axis=1)
df.head()

,Name,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,countN
13154400,lyft,1,1,0,3,4,True,15,40.864294,-73.84651,12.000000
13154401,lyft,1,1,1,3,4,True,22,40.864294,-73.84651,11.666667
13154402,lyft,1,1,2,3,4,True,17,40.864294,-73.84651,11.333333
13154403,lyft,1,1,3,3,4,True,16,40.864294,-73.84651,8.333333
13154404,lyft,1,1,4,3,4,True,12,40.864294,-73.84651,8.333333


In [5]:
valid = pd.read_parquet('../time_series/TS6/TS6-2023.parquet', engine='pyarrow')
valid = valid.drop(['year', 'TempTime', 'countN'], axis=1)
valid.head()

,Name,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon
26872560,lyft,1,1,0,3,6,True,12,40.864294,-73.84651
26872561,lyft,1,1,1,3,6,True,28,40.864294,-73.84651
26872562,lyft,1,1,2,3,6,True,19,40.864294,-73.84651
26872563,lyft,1,1,3,3,6,True,26,40.864294,-73.84651
26872564,lyft,1,1,4,3,6,True,17,40.864294,-73.84651


In [6]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
df['Name'] = label.fit_transform(df['Name'])
valid['Name'] = label.fit_transform(valid['Name'])

In [7]:
df = df.astype({'is_holiday': 'int'})
valid = valid.astype({'is_holiday': 'int'})

In [8]:
from sklearn.model_selection import train_test_split

X = df.copy(deep=True)
y = df.copy(deep=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=1)

X_train = X_train.drop(['count', 'countN'], axis=1).values
X_test = X_test.drop(['count', 'countN'], axis=1).values
y_train = y_train['countN'].values
y_test = y_test['count'].values



X_valid = valid.drop('count', axis=1).values
y_valid = valid['count'].values

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10974528, 9)
(10974528,)
(2743632, 9)
(2743632,)


## scikit-learn

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, RandomizedSearchCV
from lightgbm import LGBMRegressor

# scaler = StandardScaler()
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
X_valid_scaled = scaler.fit_transform(X_valid)

## LightGBM

In [29]:
start_time = time.time()


kf = KFold(n_splits=3, shuffle=True, random_state=0)
param_grid = {'n_estimators': [100, 1000, 10000], 
              'learning_rate': np.logspace(-1, -2, 2), 
              'max_depth': range(5, 11)}

lgbm = LGBMRegressor(subsample=0.8, 
                     n_jobs=-1, 
                     random_state=0, 
                     verbose=-1)

model_cv = RandomizedSearchCV(lgbm, param_grid, cv=kf, scoring='r2', n_iter=10)
model_cv.fit(X_train_scaled, y_train)
print(model_cv.best_params_, model_cv.best_score_)


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

{'n_estimators': 10000, 'max_depth': 10, 'learning_rate': 0.1} 0.9609267766244717
執行時間: 22858.89630484581 秒


In [ ]:
# 預設
# n_estimators=100
# learning_rate=0.1
# max_depth=-1
# num_leaves=31
# +
# subsample=0.8, 
# n_jobs=-1, 
# random_state=0, 
# verbose=-1



# 自訂
# model = LGBMRegressor(n_estimators=10000, 
#                       learning_rate=0.01, 
#                       max_depth=10, 
#                       n_jobs=-1, 
#                       random_state=0, 
#                       verbose=-1)

In [11]:
start_time = time.time()


model = LGBMRegressor(n_estimators=1000,
                      learning_rate=0.1,
                      max_depth=10,
                      num_leaves=31,
                      subsample=0.8, 
                      n_jobs=-1, 
                      random_state=0, 
                      verbose=-1)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Train')
print(f'MAE: {mae}\nR2: {r2}')
print('-' * 30)


prediction = model.predict(X_valid_scaled)
mae = mean_absolute_error(y_valid, prediction)
r2 = r2_score(y_valid, prediction)
print('Valid')
print(f'MAE: {mae}\nR2: {r2}')
                                                                                                                                                                              

end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

Train
MAE: 8.330686992463187
R2: 0.8956942934168776
------------------------------
Valid
MAE: 10.564724760083212
R2: 0.8575427096251164
執行時間: 165.48674488067627 秒


In [12]:
import pickle

with open('lgbm.pickle', 'wb') as f:
    pickle.dump(model, f)

## NN

In [15]:
start_time = time.time()

import tensorflow as tf
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras.regularizers import L1
tf.random.set_seed(42)

n_cols = X_train_scaled.shape[1]

model = Sequential([layers.Dense(64, input_shape=(n_cols,)),
                    layers.BatchNormalization(),
                    layers.LeakyReLU(),
                    layers.Dropout(0.2),
                    layers.Dense(32, kernel_regularizer=L1(0.01), input_shape=(n_cols,)),
                    layers.BatchNormalization(),
                    layers.LeakyReLU(),
                    layers.Dropout(0.2),
                    layers.Dense(16, kernel_regularizer=L1(0.01), input_shape=(n_cols,)),
                    layers.BatchNormalization(),
                    layers.LeakyReLU(),
                    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
model.summary()


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                640       
                                                                 
 batch_normalization_2 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 64)                0         
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 batch_normalization_3 (Bat  (None, 32)                128       
 chNormalization)                                     

In [16]:
start_time = time.time()


from keras.callbacks import EarlyStopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train, 
                    validation_split=0.2, 
                    batch_size=1000, 
                    epochs=100, 
                    verbose=1)# , callbacks=early_stopping

end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

Epoch 1/100
4571/4571 [==============================] - 27s 6ms/step - loss: 26.7462 - mae: 26.3840 - val_loss: 12.9536 - val_mae: 12.6172
Epoch 2/100
4571/4571 [==============================] - 26s 6ms/step - loss: 23.5265 - mae: 23.1814 - val_loss: 13.0343 - val_mae: 12.6765
Epoch 3/100
4571/4571 [==============================] - 26s 6ms/step - loss: 23.1168 - mae: 22.7486 - val_loss: 12.6915 - val_mae: 12.3140
Epoch 4/100
4571/4571 [==============================] - 26s 6ms/step - loss: 22.9285 - mae: 22.5438 - val_loss: 12.1812 - val_mae: 11.7875
Epoch 5/100
4571/4571 [==============================] - 25s 5ms/step - loss: 22.8188 - mae: 22.4189 - val_loss: 11.5539 - val_mae: 11.1539
Epoch 6/100
4571/4571 [==============================] - 25s 5ms/step - loss: 22.7320 - mae: 22.3250 - val_loss: 12.4638 - val_mae: 12.0514
Epoch 7/100
4571/4571 [==============================] - 24s 5ms/step - loss: 22.6722 - mae: 22.2576 - val_loss: 11.9838 - val_mae: 11.5653
Epoch 8/100
4571/457

KeyboardInterrupt: 

In [12]:
result = model.evaluate(X_test_scaled, y_test)
print(result)

35823/35823 [==============================] - 26s 713us/step - loss: 22.1823 - mae: 21.8188
[22.18230628967285, 21.818830490112305]


In [13]:
from sklearn.metrics import r2_score

y_test_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, y_test_pred)

print("Test R²:", r2)

35823/35823 [==============================] - 25s 703us/step
Test R²: 0.42393932020142644


In [14]:
y_valid_pred = model.predict(X_valid_scaled)
r2 = r2_score(y_valid, y_valid_pred)

print("Valid R²:", r2)

106293/106293 [==============================] - 63s 591us/step
Valid R²: 0.42111914137032913
